In [ ]:
import time

# 初始化PID参数
kp = 0.8  # 比例系数
ki = 0.05  # 积分系数
kd = 0.2  # 微分系数
pid_integral = 0
last_error = 0
last_time = time.time()

# 电机控制参数
base_duty = 60  # 基础占空比
max_duty = 100  # 最大占空比
min_duty = 30  # 最小占空比
target_speed = 50  # 目标转速(rpm)

# 初始化板卡和编码器
board.set_encoder_enable(board.ALL)
board.set_encoder_reduction_ratio(board.ALL, 43)
board.set_moter_pwm_frequency(1000)


def ramp_up(target_duty=60, duration=2):
    """缓慢加速到目标占空比"""
    print("正在加速...")
    for duty in range(0, target_duty, 5):
        board.motor_movement([board.M1], board.CW, duty)
        board.motor_movement([board.M2], board.CCW, duty)
        time.sleep(duration / target_duty * 5)


def compute_pid(error):
    """计算PID调整值"""
    global pid_integral, last_error, last_time

    current_time = time.time()
    dt = current_time - last_time

    # 比例项
    proportional = kp * error

    # 积分项（抗积分饱和）
    pid_integral += error * dt
    integral = ki * pid_integral

    # 微分项
    derivative = kd * (error - last_error) / dt if dt > 0 else 0

    # 更新状态
    last_error = error
    last_time = current_time

    return proportional + integral + derivative


# 主程序
try:
    print(f"开始直线运动，目标速度: {target_speed}rpm")

    # 缓慢加速
    ramp_up(base_duty)

    start_time = time.time()
    last_print = start_time
    duration_sec = 10  # 运行10秒

    while time.time() - start_time < duration_sec:
        # 获取当前电机转速
        speeds = board.get_encoder_speed(board.ALL)
        m1_speed, m2_speed = speeds[0], speeds[1]

        # 计算速度差 (M1作为基准，M2需要匹配M1)
        speed_error = m1_speed - m2_speed

        # 获取PID调整值
        adjustment = compute_pid(speed_error)

        # 调整电机占空比
        m1_duty = base_duty
        m2_duty = base_duty + adjustment

        # 限制占空比在合理范围内
        m2_duty = max(min_duty, min(max_duty, m2_duty))

        # 设置电机
        board.motor_movement([board.M1], board.CW, int(m1_duty))
        board.motor_movement([board.M2], board.CCW, int(m2_duty))

        # 定期打印状态
        if time.time() - last_print > 0.5:
            print(f"M1: {m1_speed:5.1f}rpm, M2: {m2_speed:5.1f}rpm, "
                  f"误差: {speed_error:5.1f}, 调整: {adjustment:5.1f}%")
            last_print = time.time()

        time.sleep(0.05)  # 控制循环频率

except KeyboardInterrupt:
    print("\n运动已手动停止")

finally:
    # 停止电机并清理
    board.motor_stop(board.ALL)
    board.set_encoder_disable(board.ALL)
    print("直线运动完成")